### To-Do
*   Delete dates that spill over to the previous year (Eg: for the 1st weekly expiry of Jan 2020, its dates spill over to 2019)



In [1]:
import pandas as pd
import numpy as np
print(pd.__version__)

1.1.0


In [2]:
# initialising google drive api
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [3]:
FILE_ID = None
QUERY = "name = 'BANKNIFTYWK16500PE.csv' and mimeType = 'text/csv'"

def getFileList(service,query=None):
  """Search for file and retrieve file id
  Args:
    service: Google Drive API service instance
  Returns:
    file id
  """
  response = []
  page_token = None
  while True:
    param = {}
    param['q'] = query
    if page_token:
      param['pageToken'] = page_token
    files = service.files().list(**param).execute()
    response.extend(files['files'])
    page_token = files.get('nextPageToken')
    if not page_token:
      break
  return response

In [4]:
import io
from googleapiclient.http import MediaIoBaseDownload

def downloadFromDrive(service,fileid):
  """Download file from Drive
  Args:
    service: Google Drive API service instance
    fileid: file id of file to be downloaded
  Returns:
    object of io class containing downloaded fie content as bytes
  """
  request = service.files().get_media(fileId=fileid)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
      status, done = downloader.next_chunk()
      print("Download %d%%." % int(status.progress() * 100))
  return downloaded

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt

close = options.Close
close.plot(linestyle='dotted')

NameError: ignored

#### Sub-Tasks


1.   Merge date and time fields as single Datetime field - **Done**
2.   Plot decay in Close value



# **Straddle**

In [5]:
# fetching stock price (for BANKNIFTY)
TICKER = 'BANKNIFTY'
TICKER_COLUMNS = ['Ticker','Date','Time','Open','High','Low','Close','Volume','OpenInterest']
ticker_data = pd.DataFrame()

ticker_query = "name = '%s' and mimeType = 'application/vnd.google-apps.folder'" % TICKER
FILE_LIST = getFileList(drive_service,ticker_query)
for file in FILE_LIST:
  FILE_ID = file['id']

filesinfolder_query = "'%s' in parents" % FILE_ID
FILE_LIST = getFileList(drive_service,filesinfolder_query)
for file in FILE_LIST:
  downloaded = downloadFromDrive(drive_service,file['id'])
  buffer_contents = downloaded.getvalue().decode(encoding='utf-8')
  data = io.StringIO(buffer_contents)
  ticker_data = ticker_data.append(pd.read_csv(data,sep=",",names=TICKER_COLUMNS,index_col=False),ignore_index=True)

ticker_data['DateTime'] = ""

Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.


In [6]:
# converting date, time values to the datetime dtype
for index in ticker_data.index:
  counter = 0
  ticker_year, ticker_month, ticker_day = '','',''
  for char in str(ticker_data['Date'][index]):
    counter += 1
    if counter<=4:
      ticker_year += char
    elif counter>4 and counter <=6:
      ticker_month += char
    else:
      ticker_day += char
  ticker_data['DateTime'][index] = pd.to_datetime(ticker_year+' '+ticker_month+' '+ticker_day+' '+ticker_data['Time'][index])

# ticker_data = ticker_data.set_index('DateTime')
ticker_data = ticker_data.drop(['Date','Time'],axis=1)
%time ticker_data.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


CPU times: user 109 µs, sys: 2 µs, total: 111 µs
Wall time: 114 µs


,Ticker,Open,High,Low,Close,Volume,OpenInterest,DateTime
0,BANKNIFTY,19122.40,19122.40,19122.40,19122.40,0,0,2020-04-01 09:08:00
1,BANKNIFTY,19149.60,19154.75,18889.80,18958.40,0,0,2020-04-01 09:16:00
2,BANKNIFTY,18953.80,18958.45,18824.75,18876.45,0,0,2020-04-01 09:17:00
3,BANKNIFTY,18878.05,18919.15,18864.00,18864.00,0,0,2020-04-01 09:18:00
4,BANKNIFTY,18867.75,18867.75,18813.75,18856.05,0,0,2020-04-01 09:19:00


In [7]:
# fetching folder for year (eg:2020)
YEAR_START = 2020
QUERY = "name = '%d' and mimeType = 'application/vnd.google-apps.folder'" % YEAR_START
FILE_LIST = getFileList(drive_service,QUERY)

for files in FILE_LIST:
  FILE_ID = files['id']

In [8]:
# fetching files in year folder
filesinfolder_query = "'%s' in parents" % FILE_ID
FILE_LIST = getFileList(drive_service,filesinfolder_query)
print(FILE_LIST)

[{'kind': 'drive#file', 'id': '1Im88FSX5sCFtoYK-CPtK4-1paPjUXbKG', 'name': 'Expiry 01st April', 'mimeType': 'application/vnd.google-apps.folder'}, {'kind': 'drive#file', 'id': '1ijoJ8f_msrefHalkzncpR2MlPQuDKCnh', 'name': 'Expiry 02nd January', 'mimeType': 'application/vnd.google-apps.folder'}, {'kind': 'drive#file', 'id': '1aoCzR6nsZbhppIz9caTXv_Ncx7mC92FK', 'name': 'Expiry 05th March', 'mimeType': 'application/vnd.google-apps.folder'}, {'kind': 'drive#file', 'id': '1txkncknlkah-RBPM3j3XodBoQix6qaSR', 'name': 'Expiry 06th February', 'mimeType': 'application/vnd.google-apps.folder'}, {'kind': 'drive#file', 'id': '1KdsuycqzhBIIMzCccwCOqdIF5DVf2aN-', 'name': 'Expiry 07th May', 'mimeType': 'application/vnd.google-apps.folder'}, {'kind': 'drive#file', 'id': '1YrfwKm3Xo4TamHL0y8tEOcZ1L6PULhTd', 'name': 'Expiry 09th April', 'mimeType': 'application/vnd.google-apps.folder'}, {'kind': 'drive#file', 'id': '1dLxx5EuOkf-Hlv5o9OWXIKV0IewkAc9N', 'name': 'Expiry 09th January', 'mimeType': 'applicatio

In [26]:
# fetching all dates when Straddle to be executed
from datetime import timedelta

dateinfile = []
allDaysForOptions = [] #to store calculable days for each weekly expiry
for file in FILE_LIST:
  dateinfile = file['name'].split(maxsplit=1)[-1]
  dateinfile += str(YEAR_START)
  rawdate = pd.to_datetime(dateinfile)
  dayCounter = rawdate.dayofweek + 1
  optionDaysForWeek = []
  insertCounter = 0
  while len(optionDaysForWeek) <= dayCounter:
    if (rawdate - timedelta(insertCounter)).dayofweek != 5 and (rawdate - timedelta(insertCounter)).dayofweek != 6: #accounting for Saturdays and Sundays
      optionDaysForWeek.append(rawdate - timedelta(insertCounter))
    insertCounter += 1
  allDaysForOptions.append(optionDaysForWeek)

In [27]:
# to delete dates that spill over to the previous year (2019)
for num in range(len(allDaysForOptions)):
  temp = [date for date in allDaysForOptions[num] if date.year == 2020]
  allDaysForOptions[num] = temp

In [29]:
# adding Date column to ticker data to make date comparisons easier
ticker_data['Date'] = ""
for index in ticker_data.index:
  ticker_data['Date'][index] = ticker_data['DateTime'][index].date()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [30]:
def getStrikePrice(option_prices,underlying_price):
  """Args
      option_prices = list of call and put option prices for a weekly expiry option
      underlying_price = market price of underlying asset at open of day
    Returns
      strike price closest to underlying_price
  """
  if option_prices:
    length = len(option_prices)
    if length>2:
      diff1 = abs(underlying_price - option_prices[length//2])
      diff2 = abs(underlying_price - option_prices[(length//2) + 1])
      if diff1<diff2:
        return getStrikePrice(option_prices[:length//2], underlying_price)
      elif diff1>diff2:
        return getStrikePrice(option_prices[(length//2)+1:], underlying_price)
      else:
        diffprime1 = abs(underlying_price - option_prices[(length//2)-1])
    else:
      diff1 = abs(underlying_price - option_prices[0])
      diff2 = abs(underlying_price - option_prices[-1])
      if diff1<diff2:
        return option_prices[0]
      elif diff2>diff1:
        return option_prices[-1]
      else:
        return option_prices[-1]

In [31]:
# NOT IN USE
a = [120,135,137,142,158,161,162,162, 189,190]
b = [16000, 16000, 16100, 16200, 16300, 16300, 16400, 16500, 16500, 16600, 16700, 16700, 16800, 16800, 16900, 16900, 17000, 17000, 17100, 17100, 17200, 17200, 17300, 17300, 17400, 17400, 17500, 17500, 17600, 17600, 17700, 17700, 17800, 17800, 17900, 17900, 18000, 18000, 18100, 18100, 18200, 18200, 18300, 18300, 18400, 18400, 18500, 18500, 18600, 18600, 18700, 18700, 18800, 18800, 18900, 18900, 19000, 19000, 19100, 19100, 19200, 19200, 19300, 19300, 19400, 19400, 19500, 19500, 19600, 19600, 19700, 19700, 19800, 19800, 19900, 19900, 20000, 20000, 20100, 20100, 20200, 20200, 20300, 20300, 20400, 20400, 20500, 20500, 20600, 20600, 20700, 20700, 20800, 20800, 20900, 20900, 21000, 21000, 21100, 21100, 21200, 21200, 21300, 21300, 21400, 21400, 21500, 21500, 21600, 21600, 21700, 21700, 21800, 21900, 22000, 22000, 22100, 22200, 22300, 22400, 22500, 22500, 22600, 22700, 22800, 22900, 23000, 23000, 23100, 23200, 23300, 23400, 23500, 23500, 23600, 23700, 23900, 24000, 24000, 24100, 24200, 24300, 24500, 24600, 24700, 24800, 25000, 25000, 25500, 25600, 26000]
c = [16000,19000,20000,20000,21000]
underlying_price = 19122
%time print(getStrikePrice(c, underlying_price))

None
CPU times: user 85 µs, sys: 2 µs, total: 87 µs
Wall time: 90.6 µs


In [32]:
# executing Straddle
for expiryDate in range(len(FILE_LIST)):
  FILE_ID = FILE_LIST[expiryDate]['id']
  for singleDate in allDaysForOptions[expiryDate]:
    underlying_price_list = ticker_data[ticker_data['Date'] == singleDate.date()]
    underlying_price_list = underlying_price_list.sort_values(by='DateTime')
    underlying_price = underlying_price_list.iloc[0]['Close']
    print('Underlying asset price for %s: %d'%(singleDate.date(),underlying_price))
    print('File ID: %s'%FILE_ID)

    #calculate the strike price closest to underlying price
    files_query = "'%s' in parents" % FILE_ID
    STRIKE_PRICE_FILE_LIST = getFileList(drive_service,files_query)
    option_prices = []
    for filename in STRIKE_PRICE_FILE_LIST:
      chars = ''
      # chars.append(char for char in filename['name'] if char.isnumeric()==True)
      for char in filename['name']:
        if char.isnumeric(): chars+=char
      option_prices.append(int(chars))
    option_prices = list(set(option_prices))
    option_prices.sort()
    print(option_prices)
    STRIKE_PRICE = getStrikePrice(option_prices, underlying_price)
    print('STRIKE PRICE :',STRIKE_PRICE)
    call_file_id = [this['id'] for this in STRIKE_PRICE_FILE_LIST if this['name']=='BANKNIFTYWK'+str(STRIKE_PRICE)+'CE.csv']
    put_file_id = [this['id'] for this in STRIKE_PRICE_FILE_LIST if this['name']=='BANKNIFTYWK'+str(STRIKE_PRICE)+'PE.csv']
    print(call_file_id, put_file_id)
    print("~~~~~~~~~~~~")
    # STRADDLE_RESULT = calcStraddle(drive_service, call_file_id, put_file_id)

Underlying asset price for 2020-04-01: 19122
File ID: 1Im88FSX5sCFtoYK-CPtK4-1paPjUXbKG
[16000, 16100, 16200, 16300, 16400, 16500, 16600, 16700, 16800, 16900, 17000, 17100, 17200, 17300, 17400, 17500, 17600, 17700, 17800, 17900, 18000, 18100, 18200, 18300, 18400, 18500, 18600, 18700, 18800, 18900, 19000, 19100, 19200, 19300, 19400, 19500, 19600, 19700, 19800, 19900, 20000, 20100, 20200, 20300, 20400, 20500, 20600, 20700, 20800, 20900, 21000, 21100, 21200, 21300, 21400, 21500, 21600, 21700, 21800, 21900, 22000, 22100, 22200, 22300, 22400, 22500, 22600, 22700, 22800, 22900, 23000, 23100, 23200, 23300, 23400, 23500, 23600, 23700, 23900, 24000, 24100, 24200, 24300, 24500, 24600, 24700, 24800, 25000, 25500, 25600, 26000]
STRIKE PRICE : 19000
['1hYTs2QYiLpHsYBOj2gIACafi0vfFSccb'] ['1EAH9XiQ5OFjdVY6HD6vqi3gcr6sjFjBU']
~~~~~~~~~~~~
Underlying asset price for 2020-03-31: 19307
File ID: 1Im88FSX5sCFtoYK-CPtK4-1paPjUXbKG
[16000, 16100, 16200, 16300, 16400, 16500, 16600, 16700, 16800, 16900, 17000

IndexError: ignored